In [1]:
import pandas as pd
from embedding import Embedding
import torch.nn as nn
import torch
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [2]:
data = pd.read_csv('C:/Users/user/OneDrive/문서/dev/UROP/Crawling/data/sample_comment.csv')
token = pd.read_csv('C:/Users/user/OneDrive/문서/dev/UROP/Crawling/tokenized_0.csv')

In [3]:
df = data[['mbti','comments']]

In [4]:
for i in range(len(df)):
    if 'E' in df.loc[i, 'mbti']:
        df.loc[i, 'mbti'] = 0
    else:
        df.loc[i, 'mbti'] = 1

C:\Users\user\AppData\Local\Temp\ipykernel_26404\2819924847.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[i, 'mbti'] = 0
C:\Users\user\AppData\Local\Temp\ipykernel_26404\2819924847.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[i, 'mbti'] = 1


In [30]:
label = [int(i) for i in df['mbti'].values.tolist()]
print(len(label))
label_tensor = torch.LongTensor(label)
print(label_tensor)

987
tensor([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0,
        0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
        0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
        1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 1, 1

In [5]:
df = df.dropna() # 결측치 왠지모르게 있어서 제거함
token = token.dropna()

In [6]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, vocab_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.hidden = self.init_hidden()
        self.embedding = nn.Embedding(vocab_size, input_size)

    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_size),
                torch.zeros(1, 1, self.hidden_size))

    def forward(self, input):
        # embedded = self.embedding(input)
        lstm_out, _ = self.lstm(input)

        # print(input.size(-1))
        # lstm_out, self.hidden = self.lstm(input.view(len(input), 1, -1), self.hidden)
        output = self.fc(lstm_out[-1])
        return output

In [7]:
em = Embedding(sereis=token['comments'])
word2vec_model = em.get_embedding_model()

[I 2023-12-10 02:42:29,197] A new study created in memory with name: no-name-fa8df275-112f-4a12-b85c-2880b8431b3b
[I 2023-12-10 02:42:29,240] Trial 0 finished with value: 32408.908203125 and parameters: {'vector_size': 13, 'window': 4, 'min_count': 26, 'sg': 0}. Best is trial 0 with value: 32408.908203125.


In [8]:
# Word2Vec 모델의 크기를 기반으로 LSTM 모델 생성
input_size = word2vec_model.vector_size
hidden_size = 128
output_size = 1
vocab_size = len(word2vec_model.wv.key_to_index)
lstm_model = LSTMModel(input_size, hidden_size, output_size, vocab_size)
# print(vocab_size)

In [9]:
input_size

13

In [10]:
# 단어에 대한 0 벡터 생성
def get_word_vector(word, model):
    try:
        return model.wv[word]
    except:
        return np.zeros(input_size)

# 모든 단어에 대한 벡터 리스트 생성
# word_vectors = [get_word_vector(word, word2vec_model) for sentence in em.corpus for word in sentence]
end = 100
word_vectors = []

for sentence in em.corpus:
    temp = []
    for word in sentence[:end]:  # 문장을 end 길이로 자르기
        temp.append(get_word_vector(word, word2vec_model))

    # 패딩 추가
    padding_length = end - len(temp)
    if padding_length > 0:
        padding = [np.zeros(word2vec_model.vector_size) for _ in range(padding_length)]
        temp.extend(padding)

    word_vectors.append(temp)

In [11]:
# word_to_index = {word: index for index, word in enumerate(word2vec_model.wv.key_to_index)}

# 단어에 대한 0 벡터 생성
def get_word_vector(word, model):
    try:
        return model.wv[word]
    except:
        return np.zeros(input_size)


# 모든 단어에 대한 벡터 리스트 생성
# word_vectors = [get_word_vector(word, word2vec_model) for sentence in em.corpus for word in sentence]
end = 100
word_vectors = []

for sentence in em.corpus:
    temp = []
    for word in sentence[:end]:  # 문장을 end 길이로 자르기
        temp.append(get_word_vector(word, word2vec_model))

    # 패딩 추가
    padding_length = end - len(temp)
    if padding_length > 0:
        padding = [np.zeros(word2vec_model.vector_size) for _ in range(padding_length)]
        temp.extend(padding)

    word_vectors.append(temp)

In [12]:
print(word_vectors[0])
print(len(word_vectors[0]))

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([ 0.05656336,  0.5858211 ,  0.47097918,  0.1843833 , -0.44536614,
        0.28965887, -0.32999063, -0.9387335 ,  0.3169385 ,  1.5504521 ,
        0.6633696 ,  0.42425624, -0.89760596], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([ 0.24189879,  0.8281523 ,  0.7856393 ,  0.40786722, -0.78088737,
        0.3031579 , -0.55746514, -1.5283772 ,  0.47553933,  2.619007  ,
        1.0720029 ,  0.7320711 , -1.4009383 ], dtype=float32), array([ 0.14056043,  0.26817933,  0.3184983 ,  0.06212727, -0.20054248,
        0.18656653, -0.19233415, -0.5544646 ,  0.18953405,  0.9310625 ,
        0.32670653,  0.28992444, -0.5077043 ], dtype=float32), array([ 0.14951816,  0.46188924,  0.36078843,  0.12612705, -0.37672815,
        0.1351262 , -0.24418435, -0.6576593 ,  0.30391207,  1.258824  ,
        0.5544954 ,  0.2801426 , -0.5910483 ], dtype=float32), array([ 0.00913632,  0.32191497,  0.3029733 ,  0.

In [13]:
for i in word_vectors:
    if len(i) != 100:
        print(i)

In [14]:
# 벡터 리스트를 텐서로 변환
input_tensor = torch.tensor(word_vectors, dtype=torch.float32)
# input_tensor = torch.LongTensor(word_vectors)

C:\Users\user\AppData\Local\Temp\ipykernel_26404\3932551150.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:204.)
  input_tensor = torch.tensor(word_vectors, dtype=torch.float32)


In [15]:
input_tensor

tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0566,  0.5858,  0.4710,  ...,  0.6634,  0.4243, -0.8976],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.1455,  0.7237,  0.6051,  ...,  0.8546,  0.5535, -1.0809],
         [ 0.1916,  0.8262,  0.7768,  ...,  0.9913,  0.6891, -1.3676],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0693,  0.2742,  0.1886,  ...,  0.3114,  0.2407, -0.4481],
         [ 0.0480,  0.5228,  0.4331,  ...,  0

In [16]:
# len(input_tensor[0])
print(input_tensor.shape)
print(input_tensor.size(-1))

torch.Size([987, 100, 13])
13


In [17]:
# LSTM 모델에 입력 전달하여 출력 받기
output = lstm_model(input_tensor)
print("LSTM 모델의 출력:", output)

LSTM 모델의 출력: tensor([[-0.0243],
        [-0.0342],
        [-0.0326],
        [-0.0374],
        [-0.0324],
        [-0.0294],
        [-0.0298],
        [-0.0307],
        [-0.0285],
        [-0.0291],
        [-0.0316],
        [-0.0360],
        [-0.0342],
        [-0.0311],
        [-0.0296],
        [-0.0350],
        [-0.0323],
        [-0.0349],
        [-0.0335],
        [-0.0342],
        [-0.0406],
        [-0.0421],
        [-0.0382],
        [-0.0335],
        [-0.0299],
        [-0.0306],
        [-0.0307],
        [-0.0341],
        [-0.0313],
        [-0.0330],
        [-0.0321],
        [-0.0360],
        [-0.0332],
        [-0.0328],
        [-0.0318],
        [-0.0353],
        [-0.0334],
        [-0.0361],
        [-0.0336],
        [-0.0344],
        [-0.0355],
        [-0.0329],
        [-0.0358],
        [-0.0320],
        [-0.0318],
        [-0.0345],
        [-0.0337],
        [-0.0315],
        [-0.0321],
        [-0.0335],
        [-0.0346],
        [-0.0384],

In [18]:
# BCE 손실 함수 정의
criterion = nn.BCEWithLogitsLoss()
# Adam 옵티마이저 설정
optimizer = optim.Adam(lstm_model.parameters(), lr=0.001)

In [19]:
class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [31]:
# data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
dataset = MyDataset(input_tensor, label_tensor)

batch_size = 32
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [32]:
# 학습 및 평가 루프 정의
def train(model, criterion, optimizer, train_loader, epochs=5):
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:  
            # train_loader에는 입력 데이터와 레이블이 있는 데이터가 포함되어야 합니다.
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())  
            # BCELoss를 사용하므로 레이블을 float로 변환합니다.
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")